In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__, "..")); Pkg.instantiate();
import Pkg; Pkg.add("RobotZoo")
using RobotZoo: YakPlane
import RobotDynamics
using RobotDynamics: state_dim, control_dim
using ForwardDiff
using StaticArrays
using LinearAlgebra
using Rotations
using Printf
using Test
using TrajOptPlots
using JLD2
const RD = RobotDynamics
include("utils.jl")
include("airplane.jl")
const isautograder = @isdefined autograder;
const resfile = joinpath(@__DIR__,"q1.jld2")

  Activating environment at `~/Desktop/OptimalC/Project/Code/Project.toml`


    Updating registry at `~/.julia/registries/General`


   Resolving package versions...


  No Changes to `~/Desktop/OptimalC/Project/Code/Project.toml`
  No Changes to `~/Desktop/OptimalC/Project/Code/Manifest.toml`


"/Users/Augusto/Desktop/OptimalC/Project/Code/src/q1.jld2"

In [2]:
function l(x,μ,point)
    if point == 1
        return x^5 - (3-μ)*x^4 + (3-2*μ)*x^3 - μ*x^2 + 2*μ*x - μ
    end
    if point == 2 
        return x^5 + (3-μ)*x^4 + (3-2*μ)*x^3 - μ*x^2 - 2*μ*x - μ
    end
end 

l (generic function with 1 method)

In [3]:
function dl(x,μ,point)
    if point == 1
        return 5*x^4 - 4*(3-μ)*x^3 + 3*(3-2*μ)*x^2 - 2*μ*x^1 + 2*μ
    end
    if point == 2 
        return 5*x^4 + 4*(3-μ)*x^3 + 3*(3-2*μ)*x^2 - 2*μ*x^1 + 2*μ
    end
end 


dl (generic function with 1 method)

In [4]:
function l_approx(μ,point)
    rh = (μ/3)^(1/3)
    if point == 1
        return rh*(1-(1/3)*rh-(1/9)*rh^2-(1/27)*rh^3)
    end
    if point == 2 
        return rh*(1+(1/3)*rh-(1/9)*rh^2-(1/27)*rh^3)
    end
end


l_approx (generic function with 1 method)

In [5]:
# TASK: Complete the following function
using ForwardDiff

function lagrange_points_solve!(x,μ,point; ϵ=1e-6)
    f = l(x,μ,point)
    e = [norm(f)]
    count = 0
    while e[end] > ϵ
        ∂f = ForwardDiff.derivative(x_ ->l(x_,μ,point), x)
        ∂f2 = ForwardDiff.derivative(x_ ->l(x_,μ,point), x)
        x = x - ∂f\f
        f = l(x,μ,point)
        append!(e,norm(f))
    end
    # TODO: Implement this function. Feel free to add optional input arguments as needed
    # residuals = Float64[]
    # residuals = implicit_midpoint(f,x1,x2)
    return e,x
end 

lagrange_points_solve! (generic function with 1 method)

In [6]:
#Sun Jupiter System 
mu = 9.537e-4
L = 1
x0_l1 = l_approx(mu,1)
x0_l2 = l_approx(mu,2)
err,γ1 = lagrange_points_solve!(x0_l1,mu,1);
err2,γ2 = lagrange_points_solve!(x0_l2,mu,2);
@show γ1
#L1 
@show 1-mu-γ1 
@show γ2
#L2
@show 1-mu+γ2
#L4
@show (L^2-(L/2)^2)^0.5 
#How do we normalize L? We just treat it as one? 

γ1 = 0.06666031474629562
(1 - mu) - γ1 = 0.9323859852537044
γ2 = 0.06976560661590613
(1 - mu) + γ2 = 1.068811906615906
(L ^ 2 - (L / 2) ^ 2) ^ 0.5 = 0.8660254037844386


0.8660254037844386